In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix,classification_report,roc_auc_score,roc_curve
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv("/kaggle/input/pima-indians-diabetes-database/diabetes.csv")
df.head()

In [ ]:
df.isnull().sum()

* **WOW! no null values are present ! but.....**

In [ ]:
df.describe().T

**As you saw in the Dataframe There are 0 values in Some Columns**
* Glucose
* BloodPressure
* SkinThickness
* Insulin
* BMI

**This is not possible as these values cannot be zero ..these are null values**

In [ ]:
diabetes = df.copy()

diabetes[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']] = diabetes[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']].replace(0,np.NaN)

**Creating a copy of data**

In [ ]:
diabetes.isnull().sum()

****Ahh !! Null values are there we will replace them by their mean or median based on their distributions shown below****

In [ ]:
df.hist(figsize=(15,15))

In [ ]:
diabetes['Glucose'].fillna(diabetes['Glucose'].median(), inplace = True)
diabetes['BloodPressure'].fillna(diabetes['BloodPressure'].mean(), inplace = True)
diabetes['SkinThickness'].fillna(diabetes['SkinThickness'].mean(), inplace = True)
diabetes['Insulin'].fillna(diabetes['Insulin'].median(), inplace = True)
diabetes['BMI'].fillna(diabetes['BMI'].median(), inplace = True)

In [ ]:
diabetes.describe().T

In [ ]:
diabetes.hist(figsize=(15,15))

**Great ! Null values are replaced now we can carry on**

In [ ]:
print(diabetes['Outcome'].value_counts())
sns.countplot(diabetes['Outcome'])

In [ ]:
plt.figure(figsize=(15,15))
corr = diabetes.corr()
sns.heatmap(corr,annot=True)

**As you can see all the features are correlated with outcome hence,we will use them all for our model**

In [ ]:
X = diabetes.drop('Outcome',axis=1)
y = diabetes['Outcome']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [ ]:
scale = MinMaxScaler()
X_train = scale.fit_transform(X_train)
X_test = scale.fit_transform(X_test)

In [ ]:
lg = LogisticRegression()
lg.fit(X_train,y_train)
lg_pred = lg.predict(X_test)

In [ ]:
p = sns.heatmap(pd.DataFrame(confusion_matrix(y_test, lg_pred)), annot=True,fmt='g')
plt.title('Confusion matrix',y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')


In [ ]:
print(classification_report(y_test,lg_pred))

**The recall rate of 0 is more as there are high in number compared to 1**

In [ ]:
y_pred_proba = lg.predict_proba(X_test)[:,1]
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)

In [ ]:
plt.plot([0,1],[0,1],'--')
plt.plot(fpr,tpr, label='LG')
plt.xlabel('fpr')
plt.ylabel('tpr')
plt.title('Logistic Regression ROC curve')
plt.show()

In [ ]:
print("The AUC score for the given model is :",roc_auc_score(y_test,y_pred_proba))

**Do Upvote the notebook if you find it useful...Share your views to improve the model in the comments section <3**